In [1]:
import time
import os
import numpy as np 
from pathlib import Path
import argparse
import ffsim
import scipy
import ffsim
import pyscf
from collections import defaultdict
from sqd_lattice.pre_process import make_pyscf_slater_det,make_two_body_tensor
from sqd_lattice.util import load_pickle,load_yaml,save_pickle,load_json,save_json
from ffsim import FermionOperator,cre_a,cre_b,des_a,des_b,linear_operator,slater_determinant,hartree_fock_state

In [2]:
material = 'platinum_2'
ne = "31"

folder_path = f"../runs/{material}/{ne}e"

# # Loading data
base_config = load_yaml(f"../runs/{material}","base_config.yaml")
data = load_pickle(f"../runs/{material}","data_dict.pkl")
results_pre_process = load_pickle(folder_path,"results_pre_process.pkl")
num_orbitals = data["hopping_matrix"].shape[0]

In [3]:
def make_hamiltonian_ffsim(data):
    # Building the Hamiltonian from scratch with fermionic operators
    coeffs: dict[tuple[tuple[bool, bool, int], ...], complex] = defaultdict(float)

    for k,v_dict in data["hubbard_params"].items():
        if k[0]==k[1]:
            for i in v_dict["index_i"]:
                coeffs[cre_a(i), des_a(i), cre_b(i), des_b(i)] = v_dict["V"]
                coeffs[cre_b(i), des_b(i), cre_a(i), des_a(i)] = v_dict["V"]
        else:
            for i in v_dict["index_i"]:
                for j in v_dict["index_j"]:
                    coeffs[cre_a(i), des_a(i), cre_a(j), des_a(j)] = v_dict["V"]
                    coeffs[cre_a(i), des_a(i), cre_b(j), des_b(j)] = v_dict["V"]
                    coeffs[cre_b(i), des_b(i), cre_a(j), des_a(j)] = v_dict["V"]
                    coeffs[cre_b(i), des_b(i), cre_b(j), des_b(j)] = v_dict["V"]

                    coeffs[cre_a(j), des_a(j), cre_a(i), des_a(i)] = v_dict["V"]
                    coeffs[cre_a(j), des_a(j), cre_b(i), des_b(i)] = v_dict["V"]
                    coeffs[cre_b(j), des_b(j), cre_a(i), des_a(i)] = v_dict["V"]
                    coeffs[cre_b(j), des_b(j), cre_b(i), des_b(i)] = v_dict["V"]

    for i in range(data["num_orbitals"]):
        for j in range(data["num_orbitals"]):
            coeffs[cre_a(i),des_a(j)] = data["hopping_matrix"][i,j]
            coeffs[cre_b(i),des_b(j)] = data["hopping_matrix"][i,j]

    return FermionOperator(coeffs)

ham_ffsim = make_hamiltonian_ffsim(data)

In [4]:
def make_two_body_tensor(data:dict) -> np.ndarray:
    num_orbitals = data["hopping_matrix"].shape[0]
    two_body_tensor = np.zeros((num_orbitals,num_orbitals,num_orbitals,num_orbitals))
    for k,v_dict in data["hubbard_params"].items():
        if k[0]==k[1]:
            for i in v_dict["index_i"]:
                two_body_tensor[i,i,i,i] = 2*v_dict["V"]
        else:
            for i in v_dict["index_i"]:
                for j in v_dict["index_j"]:
                    two_body_tensor[i,i,j,j] = 2*v_dict["V"]
                    two_body_tensor[j,j,i,i] = 2*v_dict["V"]
    
    return two_body_tensor

two_body_tensor = make_two_body_tensor(data)

from sqd_lattice.pre_process import rotate_tensors

h1,h2 = rotate_tensors(data["hopping_matrix"],two_body_tensor,results_pre_process["mo_coeff"])

ham_pyscf = ffsim.MolecularHamiltonian(one_body_tensor=data["hopping_matrix"],two_body_tensor=two_body_tensor)
# ham_pyscf = ffsim.MolecularHamiltonian(one_body_tensor=results_pre_process["hopping_matrix_rot"],two_body_tensor=results_pre_process["two_body_tensor_rot"])
# ham_pyscf = ffsim.MolecularHamiltonian(one_body_tensor=h1,two_body_tensor=h2)
# ham_pyscf = ham_pyscf.rotated(results_pre_process["mo_coeff"])

In [5]:
lin_op = linear_operator(ham_ffsim,num_orbitals,results_pre_process["ne_ab"])
state = hartree_fock_state(num_orbitals,results_pre_process["ne_ab"])

# occupied_orbitals_a = [0,  3,  4,  2, 14, 10,  1,  8,  9,  5, 15, 11,  6,  7, 12]
# occupied_orbitals_b = [0,  3,  4,  2, 14, 10,  1,  8,  9,  5, 15, 11,  6,  7, 12]
# occupied_orbitals = [occupied_orbitals_a,occupied_orbitals_b]
# state = slater_determinant(num_orbitals,occupied_orbitals=occupied_orbitals)

energy = np.real(np.vdot(state, lin_op @ state))
energy

np.float64(-580.277236777386)

In [6]:
lin_op = linear_operator(ham_pyscf,num_orbitals,results_pre_process["ne_ab"])

state = hartree_fock_state(num_orbitals,nelec=results_pre_process["ne_ab"])

energy = np.real(np.vdot(state, lin_op @ state))

energy

np.float64(-580.2772367773837)